Transformation is done based on this document -- [Link](https://docs.google.com/document/d/1_wt-wO18sa5iQBy3b-WpSHDjCqRl3MeN67N1v3dTZtQ/edit#heading=h.usgit3agfa62)

In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib

In [2]:
#Commented out until a shapefile for HP village boudaries can be acquired
#bharat_maps_villages = gpd.read_file(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Himachal-Pradesh\Maps\HP\DDD_HP_villageboundary.geojson')

In [3]:
mission_antyodaya_df = pd.read_excel(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Himachal-Pradesh\Sources\ANTYODAYA\data\HP_antyodaya_village_dataset.xlsx')

In [4]:
mission_antyodaya_df.head()

,state_code,state_name,district_code,district_name,sub_district_code,sub_district_name,block_code,block_name,gp_code,gp_name,...,total_no_of_farmers_add_fert_in_soil_as_per_report,total_no_of_elected_representatives,total_no_of_elect_rep_oriented_under_rgsa,total_no_of_elect_rep_undergone_training_under_rgsa,total_approved_labour_budget_for_year,total_expenditure_approved_under_nrm_labour_budget_during_yr,total_area_covered_under_irrigation_using_drip_sprinkler,total_hhd_having_piped_water_connection,village_latitude,village_longitude
0,2,HIMACHAL PRADESH,25,SOLAN,158,Arki,202,KUNIHAR,10183,BHUMTI,...,9,1,0,0,2.0,0.0,0.0,44,31.117117,76.924280
1,2,HIMACHAL PRADESH,25,SOLAN,158,Arki,202,KUNIHAR,10183,BHUMTI,...,0,0,0,0,0.0,0.0,0.0,1,31.117117,76.924280
2,2,HIMACHAL PRADESH,18,KANGRA,109,Dhira,164,SULAH,8836,KAHANPAT,...,0,1,0,0,50000.0,50000.0,0.0,21,32.018017,76.459785
3,2,HIMACHAL PRADESH,25,SOLAN,158,Arki,202,KUNIHAR,10205,PALOG,...,0,0,0,0,0.0,0.0,0.0,4,31.084871,77.033690
4,2,HIMACHAL PRADESH,25,SOLAN,158,Arki,202,KUNIHAR,10205,PALOG,...,0,0,0,0,0.0,0.0,0.0,18,31.084877,77.033676


In [15]:
mission_antyodaya_df['sub_district_code'].value_counts()

sub_district_code
181     482
165     481
139     398
184     395
143     332
       ... 
6953     16
186      14
6886     10
193       9
7048      2
Name: count, Length: 172, dtype: int64

In [277]:
mission_antyodaya_edited = mission_antyodaya_df[['sub_district_code','district_name', 'sub_district_name', 'net_sown_area_in_hac', 
                             'total_hhd', 'total_hhd_having_piped_water_connection', 
                             'total_hhd_not_having_sanitary_latrines', 
                             'availablility_hours_of_domestic_electricity', 
                             'availability_of_telephone_services']]

# Convert to numeric columns that should be summed or averaged
columns_to_sum = ['net_sown_area_in_hac', 'total_hhd', 'total_hhd_having_piped_water_connection', 'total_hhd_not_having_sanitary_latrines']
columns_to_avg = ['availablility_hours_of_domestic_electricity', 'availability_of_telephone_services']

mission_antyodaya_edited[columns_to_sum] = mission_antyodaya_edited[columns_to_sum].apply(pd.to_numeric, errors='coerce')
mission_antyodaya_edited[columns_to_avg] = mission_antyodaya_edited[columns_to_avg].apply(pd.to_numeric, errors='coerce')

# Group by 'sub_district_code' and merge accordingly
aggregated_df = mission_antyodaya_edited.groupby('sub_district_code').agg({
    'sub_district_name': 'first',
    'district_name': 'first',
    'net_sown_area_in_hac': 'sum',
    'total_hhd': 'sum',
    'total_hhd_having_piped_water_connection': 'sum',
    'total_hhd_not_having_sanitary_latrines': 'sum',
    'availablility_hours_of_domestic_electricity': 'mean',
    'availability_of_telephone_services': 'mean'
}).reset_index()

aggregated_df.head()

C:\Users\saura\AppData\Local\Temp\ipykernel_17496\3933786443.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mission_antyodaya_edited[columns_to_sum] = mission_antyodaya_edited[columns_to_sum].apply(pd.to_numeric, errors='coerce')
C:\Users\saura\AppData\Local\Temp\ipykernel_17496\3933786443.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mission_antyodaya_edited[columns_to_avg] = mission_antyodaya_edited[columns_to_avg].apply(pd.to_numeric, errors='coerce')


,sub_district_code,sub_district_name,district_name,net_sown_area_in_hac,total_hhd,total_hhd_having_piped_water_connection,total_hhd_not_having_sanitary_latrines,availablility_hours_of_domestic_electricity,availability_of_telephone_services
0,83,Pangi,CHAMBA,27219.770,6617,1195,1049,2.754717,3.632075
1,84,Chaurah,CHAMBA,20584.072,20801,5738,1506,4.100334,2.979933
2,85,Saluni,CHAMBA,1929.490,11175,2908,124,4.015267,2.396947
3,86,Bhalai,CHAMBA,2899.230,6639,3274,228,4.012821,2.256410
4,87,Dalhousie,CHAMBA,1570.300,10934,4414,570,3.974576,2.415254


In [280]:
aggregated_df.to_csv(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Himachal-Pradesh\Sources\ANTYODAYA\data\variables\antyodaya/MissionAntyodaya2020_HP_vul.csv', index=False)

In [147]:
mission_antyodaya_geometry = [
        Point(xy) for xy in zip(
            mission_antyodaya_df.village_longitude,
            mission_antyodaya_df.village_latitude
        )
    ]

In [148]:
gdf_points = GeoDataFrame(mission_antyodaya_df, geometry=mission_antyodaya_geometry)

In [9]:
gdf_polygons = GeoDataFrame.from_file(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Himachal-Pradesh\Maps\HP\HP\BharatMaps_HP_subdistricts.geojson')

In [142]:
gdf_polygons

,objectid,stcode11,dtcode11,sdtcode11,stname,dtname,sdtname,subdt_lgd,dist_lgd,state_lgd,st_area_sh,st_length_,remarks,ac_no,test,st_area(shape),st_length(shape),geometry
0,1,02,023,00088,HIMACHAL PRADESH,Chamba,Bhattiyat,88,16,2,0,0,,5,2,3.695661e+08,101900.165772,"POLYGON ((76.03658 32.52185, 76.03503 32.52221..."
1,2,02,023,00092,HIMACHAL PRADESH,Chamba,Bharmour,92,16,2,0,0,,2,3,1.334739e+09,239400.256000,"POLYGON ((76.78355 32.54782, 76.78268 32.54783..."
2,3,02,023,00090,HIMACHAL PRADESH,Chamba,Chamba,90,16,2,0,0,,3,4,1.422634e+09,195066.808024,"POLYGON ((76.09595 32.70255, 76.09575 32.70273..."
3,4,02,023,00084,HIMACHAL PRADESH,Chamba,Churah,84,16,2,0,0,,1,5,1.652463e+09,224217.320127,"POLYGON ((76.22226 33.02277, 76.22149 33.02291..."
4,5,02,023,00087,HIMACHAL PRADESH,Chamba,Dalhousie,87,16,2,0,0,,4,6,2.287591e+08,85680.411712,"POLYGON ((75.92925 32.61495, 75.92705 32.61633..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,3317,02,033,00183,HIMACHAL PRADESH,Shimla,Junga,183,23,2,0,0,,62,3318,1.504535e+08,63973.266994,"POLYGON ((77.30567 31.00461, 77.30566 31.00462..."
119,3318,02,032,00176,HIMACHAL PRADESH,Sirmaur,Ronhat,176,24,2,0,0,,59,3319,1.341459e+08,63052.419007,"POLYGON ((77.62574 30.80987, 77.62508 30.81002..."
120,3396,02,027,00130,HIMACHAL PRADESH,Mandi,Bhadrota,130,22,2,0,0,,35,3397,1.077063e+08,58588.320611,"POLYGON ((76.77496 31.62199, 76.77497 31.62199..."
121,3397,02,029,00147,HIMACHAL PRADESH,Una,Bharwain,147,26,2,0,0,,41,3398,1.361356e+08,101878.868579,"POLYGON ((76.06412 31.84618, 76.06391 31.84622..."


In [279]:
()
gdf_polygons.rename(columns={'subdt_lgd': 'sub_district_code'}, inplace=True)
gdf_polygons.rename(columns={'sdtname': 'sub_district_name'}, inplace=True)
gdf_polygons.rename(columns={'dtname': 'district_name'}, inplace=True)

In [224]:
gdf_polygons

,objectid,stcode11,dtcode11,sdtcode11,stname,dtname,sub_district_name,sub_district_code,dist_lgd,state_lgd,st_area_sh,st_length_,remarks,ac_no,test,st_area(shape),st_length(shape),geometry
0,1,02,023,00088,HIMACHAL PRADESH,Chamba,Bhattiyat,88,16,2,0,0,,5,2,3.695661e+08,101900.165772,"POLYGON ((76.03658 32.52185, 76.03503 32.52221..."
1,2,02,023,00092,HIMACHAL PRADESH,Chamba,Bharmour,92,16,2,0,0,,2,3,1.334739e+09,239400.256000,"POLYGON ((76.78355 32.54782, 76.78268 32.54783..."
2,3,02,023,00090,HIMACHAL PRADESH,Chamba,Chamba,90,16,2,0,0,,3,4,1.422634e+09,195066.808024,"POLYGON ((76.09595 32.70255, 76.09575 32.70273..."
3,4,02,023,00084,HIMACHAL PRADESH,Chamba,Churah,84,16,2,0,0,,1,5,1.652463e+09,224217.320127,"POLYGON ((76.22226 33.02277, 76.22149 33.02291..."
4,5,02,023,00087,HIMACHAL PRADESH,Chamba,Dalhousie,87,16,2,0,0,,4,6,2.287591e+08,85680.411712,"POLYGON ((75.92925 32.61495, 75.92705 32.61633..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,3317,02,033,00183,HIMACHAL PRADESH,Shimla,Junga,183,23,2,0,0,,62,3318,1.504535e+08,63973.266994,"POLYGON ((77.30567 31.00461, 77.30566 31.00462..."
119,3318,02,032,00176,HIMACHAL PRADESH,Sirmaur,Ronhat,176,24,2,0,0,,59,3319,1.341459e+08,63052.419007,"POLYGON ((77.62574 30.80987, 77.62508 30.81002..."
120,3396,02,027,00130,HIMACHAL PRADESH,Mandi,Bhadrota,130,22,2,0,0,,35,3397,1.077063e+08,58588.320611,"POLYGON ((76.77496 31.62199, 76.77497 31.62199..."
121,3397,02,029,00147,HIMACHAL PRADESH,Una,Bharwain,147,26,2,0,0,,41,3398,1.361356e+08,101878.868579,"POLYGON ((76.06412 31.84618, 76.06391 31.84622..."


In [156]:
gdf_points.crs = gdf_polygons.crs

In [157]:
result = gpd.sjoin(gdf_points, gdf_polygons, how="left", op="within")

d:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\.conda\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [158]:
result.tail()

,state_code,state_name,district_code,district_name,sub_district_code,sub_district_name,block_code,block_name,gp_code,gp_name,...,subdt_lgd,dist_lgd,state_lgd,st_area_sh,st_length_,remarks,ac_no,test,st_area(shape),st_length(shape)
19153,2,HIMACHAL PRADESH,18,KANGRA,110,Jaisinghpur,157,LAMBAGAON,8503,RIT (GUGGA),...,110.0,18.0,2.0,0.0,0.0,,13.0,2120.0,2.660055e+08,115337.042400
19154,2,HIMACHAL PRADESH,18,KANGRA,110,Jaisinghpur,157,LAMBAGAON,8503,RIT (GUGGA),...,110.0,18.0,2.0,0.0,0.0,,13.0,2120.0,2.660055e+08,115337.042400
19155,2,HIMACHAL PRADESH,18,KANGRA,6897,Chadiyar,157,LAMBAGAON,8491,KUDANG,...,112.0,18.0,2.0,0.0,0.0,,20.0,2111.0,4.089918e+08,128549.762319
19156,2,HIMACHAL PRADESH,18,KANGRA,110,Jaisinghpur,157,LAMBAGAON,8489,JANGAL,...,140.0,17.0,2.0,0.0,0.0,,37.0,2861.0,2.381732e+08,102588.129238
19157,2,HIMACHAL PRADESH,18,KANGRA,6900,Alampur,157,LAMBAGAON,8489,JANGAL,...,140.0,17.0,2.0,0.0,0.0,,37.0,2861.0,2.381732e+08,102588.129238


In [159]:
result.shape

(19158, 217)

In [ ]:
'''
result[['gp_code','village_code','sub_district_code','sdtname','net_sown_area_in_hac','total_hhd',
        'availablility_hours_of_domestic_electricity', 'availability_of_telephone_services',
       'total_hhd_having_piped_water_connection', 'total_hhd_not_having_sanitary_latrines']].to_csv(
    '/home/krishna/IDS-DRR-Data-Pipeline/Sources/ANTYODAYA/data/MissionAntyodaya2020_Assam_vul.csv', index=False)

## Note: Data for urban settlements needs to be added to the variable calculations

# Vul variables

In [349]:
vul_df = pd.read_csv(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Himachal-Pradesh\Sources\ANTYODAYA\data\variables\antyodaya/MissionAntyodaya2020_HP_vul.csv')

In [350]:
#vul_df = vul_df.rename(columns={'total_hhd':'rural_hhd'})

## net_sown_area_in_hac

In [351]:
net_sown_area_in_hac_rc = vul_df.groupby(['sub_district_code'])[['net_sown_area_in_hac']].sum()

In [352]:
net_sown_area_in_hac_rc

,net_sown_area_in_hac
sub_district_code,
83,27219.770
84,20584.072
85,1929.490
86,2899.230
87,1570.300
...,...
6967,1341.610
6969,1213.287
6970,1003.190


## Rural - Urban Households in each rc

In [353]:
total_hhd = vul_df.groupby('sub_district_code')[['total_hhd']].sum().reset_index()
total_hhd

,sub_district_code,total_hhd
0,83,6617
1,84,20801
2,85,11175
3,86,6639
4,87,10934
...,...,...
167,6967,3387
168,6969,6192
169,6970,4429
170,7048,287


In [354]:
vul_df[vul_df.sub_district_code == 6919]

,sub_district_code,sub_district_name,district_name,net_sown_area_in_hac,total_hhd,total_hhd_having_piped_water_connection,total_hhd_not_having_sanitary_latrines,availablility_hours_of_domestic_electricity,availability_of_telephone_services
137,6919,Majheen,KANGRA,868.02,3050,2009,251,3.666667,2.229167


In [355]:
#total_hhd = urban_hhd.merge(rural_hhd, how='outer', on='object_id')
#total_hhd

In [356]:
#total_hhd = total_hhd.fillna(0)
#total_hhd['total_hhd']=total_hhd['rural_hhd']+total_hhd['urban_hhd']

In [357]:
total_hhd

,sub_district_code,total_hhd
0,83,6617
1,84,20801
2,85,11175
3,86,6639
4,87,10934
...,...,...
167,6967,3387
168,6969,6192
169,6970,4429
170,7048,287


## availablility_hours_of_domestic_electricity

In [358]:
# Map category to mean electricity available
electricity_dict = {1:2.5, 2:6, 3:10, 4:12, 5:0}

In [359]:
elect_df = vul_df.replace({"availablility_hours_of_domestic_electricity": electricity_dict})[['sub_district_code','total_hhd','availablility_hours_of_domestic_electricity']]

In [360]:
#Add comment to explain
elect_df['total_electricity'] = elect_df['total_hhd']*elect_df['availablility_hours_of_domestic_electricity']

In [361]:
elect_df = elect_df.groupby('sub_district_code')[['total_hhd','total_electricity']].sum().reset_index()

In [362]:
elect_df

,sub_district_code,total_hhd,total_electricity
0,83,6617,18227.962264
1,84,20801,85291.056856
2,85,11175,44870.610687
3,86,6639,26641.115385
4,87,10934,43458.016949
...,...,...,...
167,6967,3387,13096.400000
168,6969,6192,24684.324324
169,6970,4429,53148.000000
170,7048,287,3444.000000


In [363]:
#to be used once urban_shape_files.shp is received from HP SDMA
#rc_electricity = elect_df.merge(urban_hhd, on='object_id', how='outer')
#replacement for subdistricts:
sd_electricity = elect_df

In [364]:
sd_electricity = sd_electricity.fillna(0)

In [365]:
#sd_electricity['total_electricity'] = sd_electricity['rural_electricity'] + sd_electricity['urban_electricity']

In [366]:
#rc_electricity['total_hhd'] = rc_electricity['rural_hhd'] + rc_electricity['urban_hhd']

In [367]:
sd_electricity

,sub_district_code,total_hhd,total_electricity
0,83,6617,18227.962264
1,84,20801,85291.056856
2,85,11175,44870.610687
3,86,6639,26641.115385
4,87,10934,43458.016949
...,...,...,...
167,6967,3387,13096.400000
168,6969,6192,24684.324324
169,6970,4429,53148.000000
170,7048,287,3444.000000


In [368]:
sd_electricity['avg_electricity'] = sd_electricity['total_electricity']/sd_electricity['total_hhd']

In [369]:
sd_electricity

,sub_district_code,total_hhd,total_electricity,avg_electricity
0,83,6617,18227.962264,2.754717
1,84,20801,85291.056856,4.100334
2,85,11175,44870.610687,4.015267
3,86,6639,26641.115385,4.012821
4,87,10934,43458.016949,3.974576
...,...,...,...,...
167,6967,3387,13096.400000,3.866667
168,6969,6192,24684.324324,3.986486
169,6970,4429,53148.000000,12.000000
170,7048,287,3444.000000,12.000000


## availability_of_telephone_services

In [370]:
telephone_df = vul_df.copy()

In [371]:
telephone_df['total_tele'] = vul_df['total_hhd']*vul_df['availability_of_telephone_services']

In [372]:
telephone_df = telephone_df[['sub_district_code', 'total_hhd', 'total_tele']]

In [373]:
telephone_df = telephone_df.groupby('sub_district_code')[['total_hhd','total_tele']].sum().reset_index()

In [374]:
sd_tele = telephone_df#.merge(total_hhd[['sub_district_code', 'total_hhd','total_tele']], on='sub_district_code', how='outer')

In [375]:
sd_tele = sd_tele.fillna(0)

In [376]:
#rc_tele['total_tele'] = rc_tele['rural_tele'] + rc_tele['urban_tele']

In [377]:
#rc_tele['total_hhd'] = rc_tele['rural_hhd'] + rc_tele['urban_hhd']

In [378]:
sd_tele['avg_tele'] = round(sd_tele['total_tele']/sd_tele['total_hhd'])

In [379]:
sd_tele

,sub_district_code,total_hhd,total_tele,avg_tele
0,83,6617,24033.443396,4.0
1,84,20801,61985.588629,3.0
2,85,11175,26785.877863,2.0
3,86,6639,14980.307692,2.0
4,87,10934,26408.389831,2.0
...,...,...,...,...
167,6967,3387,8213.475000,2.0
168,6969,6192,15145.297297,2.0
169,6970,4429,10376.514286,2.0
170,7048,287,717.500000,2.0


## total_hhd_having_piped_water_connection

In [380]:
pipe_df = vul_df.copy()

In [381]:
pipe_df = pipe_df.groupby('sub_district_code')[['total_hhd_having_piped_water_connection']].sum().reset_index()

In [382]:
#pipe_df = pipe_df.merge(total_hhd[['sub_district_code','total_hhd_pipe']], on='sub_district_code', how='outer')

In [383]:
pipe_df = pipe_df.fillna(0)

In [384]:
pipe_df['sd_piped_hhds'] = pipe_df['total_hhd_having_piped_water_connection'] #+ pipe_df['total_hhd_pipe']

In [385]:
pipe_df = pipe_df.merge(total_hhd[['sub_district_code','total_hhd']], on='sub_district_code')

In [386]:
pipe_df['sd_piped_hhds_pct'] = 100*pipe_df['sd_piped_hhds']/pipe_df['total_hhd']

In [387]:
pipe_df

,sub_district_code,total_hhd_having_piped_water_connection,sd_piped_hhds,total_hhd,sd_piped_hhds_pct
0,83,1195,1195,6617,18.059544
1,84,5738,5738,20801,27.585212
2,85,2908,2908,11175,26.022371
3,86,3274,3274,6639,49.314656
4,87,4414,4414,10934,40.369490
...,...,...,...,...,...
167,6967,298,298,3387,8.798347
168,6969,3028,3028,6192,48.901809
169,6970,2367,2367,4429,53.443215
170,7048,287,287,287,100.000000


## total_hhd_not_having_sanitary_latrines

In [388]:
nosanitation_df = vul_df.copy()

In [389]:
nosanitation_df = nosanitation_df.groupby('sub_district_code')[['total_hhd_not_having_sanitary_latrines']].sum().reset_index()

In [390]:
#nosanitation_df = nosanitation_df.merge(urban_hhd[['sub_district_code','total_no_sanitation']], on='object_id', how='outer')

In [391]:
nosanitation_df = nosanitation_df.fillna(0)

In [392]:
nosanitation_df['sd_nosanitation_hhds'] = nosanitation_df['total_hhd_not_having_sanitary_latrines'] # + nosanitation_df['urban_no_sanitation'] + 

In [393]:
nosanitation_df = nosanitation_df.merge(total_hhd[['sub_district_code','total_hhd']], on='sub_district_code')

In [394]:
nosanitation_df['sd_nosanitation_hhds_pct'] = 100*nosanitation_df['sd_nosanitation_hhds']/nosanitation_df['total_hhd']

In [395]:
nosanitation_df

,sub_district_code,total_hhd_not_having_sanitary_latrines,sd_nosanitation_hhds,total_hhd,sd_nosanitation_hhds_pct
0,83,1049,1049,6617,15.853106
1,84,1506,1506,20801,7.240037
2,85,124,124,11175,1.109620
3,86,228,228,6639,3.434252
4,87,570,570,10934,5.213097
...,...,...,...,...,...
167,6967,166,166,3387,4.901092
168,6969,186,186,6192,3.003876
169,6970,160,160,4429,3.612554
170,7048,1,1,287,0.348432


# ANTYODAYA MASTER

In [396]:
antyodaya_master_df = net_sown_area_in_hac_rc.reset_index()

In [397]:
antyodaya_master_df = antyodaya_master_df.merge(sd_electricity[['sub_district_code', 'avg_electricity']], how='outer')

In [398]:
antyodaya_master_df = antyodaya_master_df.merge(sd_tele[['sub_district_code', 'avg_tele']])

In [399]:
antyodaya_master_df = antyodaya_master_df.merge(pipe_df[['sub_district_code', 'sd_piped_hhds_pct']])

In [400]:
antyodaya_master_df = antyodaya_master_df.merge(nosanitation_df[['sub_district_code','sd_nosanitation_hhds_pct']])


In [401]:
antyodaya_master_df

,sub_district_code,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct
0,83,27219.770,2.754717,4.0,18.059544,15.853106
1,84,20584.072,4.100334,3.0,27.585212,7.240037
2,85,1929.490,4.015267,2.0,26.022371,1.109620
3,86,2899.230,4.012821,2.0,49.314656,3.434252
4,87,1570.300,3.974576,2.0,40.369490,5.213097
...,...,...,...,...,...,...
167,6967,1341.610,3.866667,2.0,8.798347,4.901092
168,6969,1213.287,3.986486,2.0,48.901809,3.003876
169,6970,1003.190,12.000000,2.0,53.443215,3.612554
170,7048,56.640,12.000000,2.0,100.000000,0.348432


In [402]:
antyodaya_master_df = pd.merge(gdf_polygons,antyodaya_master_df,on='sub_district_code', how='outer')

In [403]:
antyodaya_master_df[antyodaya_master_df.avg_electricity.isnull()]

,objectid,stcode11,dtcode11,sdtcode11,stname,district_name,sub_district_name,sub_district_code,dist_lgd,state_lgd,...,ac_no,test,st_area(shape),st_length(shape),geometry,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct
99,3316.0,02,033,00182,HIMACHAL PRADESH,Shimla,Shimla (urban),182,23.0,2.0,...,63.0,3317.0,2.384353e+07,31992.99194,"POLYGON ((77.16427 31.11695, 77.16365 31.11734...",NaN,NaN,NaN,NaN,NaN


In [404]:
antyodaya_master_df = antyodaya_master_df.merge(total_hhd[['sub_district_code', 'total_hhd']], on='sub_district_code')
antyodaya_master_df['total_hhd'] = antyodaya_master_df['total_hhd'].apply(lambda x: round(x,0))

In [405]:

sub_district_names_mapping = vul_df.set_index('sub_district_code')['sub_district_name'].to_dict()
# Map the sub_district_names back to the merged DataFrame
antyodaya_master_df['sub_district_name'] = antyodaya_master_df['sub_district_code'].map(sub_district_names_mapping)

# Check if there are any missing sub_district_names
missing_names = antyodaya_master_df['sub_district_name'].isna().sum()
print(f"Number of missing sub_district_names: {missing_names}")

Number of missing sub_district_names: 0


In [406]:

district_names_mapping = vul_df.set_index('sub_district_code')['district_name'].to_dict()
# Map the sub_district_names back to the merged DataFrame
antyodaya_master_df['district_name'] = antyodaya_master_df['sub_district_code'].map(district_names_mapping)

# Check if there are any missing sub_district_names
missing_names = antyodaya_master_df['district_name'].isna().sum()
print(f"Number of missing district_names: {missing_names}")

Number of missing district_names: 0


In [407]:
antyodaya_export = antyodaya_master_df.drop(['geometry','st_area(shape)', 'st_length(shape)','dist_lgd','sdtcode11','dtcode11','remarks','ac_no' ,'test','state_lgd','st_area_sh','st_length_' , 'stcode11','stname', 'objectid'                         
                           ],axis=1)
antyodaya_export.round(2).to_csv(r'D:\CivicDataLabs_IDS-DRR\IDS-DRR_Github\flood-data-ecosystem-Himachal-Pradesh\Sources\ANTYODAYA\data\variables\antyodaya/antyodaya_variables.csv', index=False)

In [408]:
antyodaya_export.shape

(172, 9)

In [409]:
antyodaya_master_df

,objectid,stcode11,dtcode11,sdtcode11,stname,district_name,sub_district_name,sub_district_code,dist_lgd,state_lgd,...,test,st_area(shape),st_length(shape),geometry,net_sown_area_in_hac,avg_electricity,avg_tele,sd_piped_hhds_pct,sd_nosanitation_hhds_pct,total_hhd
0,6.0,02,023,00083,HIMACHAL PRADESH,CHAMBA,Pangi,83,16.0,2.0,...,7.0,2.154975e+09,227109.042007,"POLYGON ((76.57266 33.20633, 76.56888 33.20689...",27219.770,2.754717,4.0,18.059544,15.853106,6617
1,4.0,02,023,00084,HIMACHAL PRADESH,CHAMBA,Chaurah,84,16.0,2.0,...,5.0,1.652463e+09,224217.320127,"POLYGON ((76.22226 33.02277, 76.22149 33.02291...",20584.072,4.100334,3.0,27.585212,7.240037,20801
2,7.0,02,023,00085,HIMACHAL PRADESH,CHAMBA,Saluni,85,16.0,2.0,...,8.0,5.078111e+08,130735.805692,"POLYGON ((75.84641 32.92777, 75.84582 32.92819...",1929.490,4.015267,2.0,26.022371,1.109620,11175
3,3312.0,02,023,00086,HIMACHAL PRADESH,CHAMBA,Bhalai,86,16.0,2.0,...,3313.0,2.037403e+08,83175.890417,"POLYGON ((76.03918 32.66771, 76.03916 32.66772...",2899.230,4.012821,2.0,49.314656,3.434252,6639
4,5.0,02,023,00087,HIMACHAL PRADESH,CHAMBA,Dalhousie,87,16.0,2.0,...,6.0,2.287591e+08,85680.411712,"POLYGON ((75.92925 32.61495, 75.92705 32.61633...",1570.300,3.974576,2.0,40.369490,5.213097,10934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,NaN,NaN,NaN,NaN,NaN,SOLAN,Mamlig,6967,NaN,NaN,...,NaN,NaN,NaN,None,1341.610,3.866667,2.0,8.798347,4.901092,3387
168,NaN,NaN,NaN,NaN,NaN,MANDI,Makridi,6969,NaN,NaN,...,NaN,NaN,NaN,None,1213.287,3.986486,2.0,48.901809,3.003876,6192
169,NaN,NaN,NaN,NaN,NaN,MANDI,Mandap,6970,NaN,NaN,...,NaN,NaN,NaN,None,1003.190,12.000000,2.0,53.443215,3.612554,4429
170,NaN,NaN,NaN,NaN,NaN,KANGRA,Sulah,7048,NaN,NaN,...,NaN,NaN,NaN,None,56.640,12.000000,2.0,100.000000,0.348432,287
